# Phân tích tập dữ liệu các trận đá bóng ở châu Âu giai đoạn 2008 - 2016

Dataset dữ liệu được lấy từ [Kaggle](https://www.kaggle.com/datasets/abdelrhmanragab/european-soccer-database) gồm các thông tin của các trận đấu, cầu thủ, đội bóng ở các nước châu Âu trong 11 giải đấu khác nhau với hơn 10 ngàn trận đấu giai đoạn 2008 - 2016. Bộ dataset này có 7 file `*.csv` khác nhau chứa các thông tin:

* `Country.csv` Các quốc gia được khảo sát
* `League.csv` Các giải đấu được khảo sát
* `Match.csv` Các trận đấu diễn tra trong giai đoạn 2008 - 2016
* `Player.csv` Thông tin các cầu thủ (tên, ngày sinh, chiều cao, cân nặng)
* `Player_Attributes.csv` Thông tin đặc tính cầu thủ
* `Team.csv` Thông tin đội bóng
* `Team_Attributes.csv` Thông tin đặc tính của đội bóng

Với tập dữ liệu này, thông qua việc khảo sát phân tích sẽ cố gắng tìm câu trả lời cho các câu hỏi sau

* Có bao nhiêu trận đấu trong mỗi giải đấu ở mùa giải 2016?
* Giải đấu nào có số trận hòa nhiều nhất trong mùa giải 2016?
* Giải đấu nào có số trận thắng nhiều nhất trong mùa giải 2016?
* Đội bóng nào để thua ít trận nhất trong mùa giải 2016?
* Đội bóng nào có số trận thắng nhiều nhất trong mùa giải 2016?
* Giải đấu nào có nhiều bàn thắng nhất trong mùa giải 2016?
* Đội bóng nào có kết quả cải thiện tốt nhất?
* Cầu thủ nào bị phạt nhiều nhất?
* Những đặc tính nào ảnh hưởng nhiều nhất nhất đến việc đội bóng giành chiến thắng?
* Có bao nhiêu cầu thủ có hiệu suất tổng thể hơn 90?
* Các đặc tính đóng góp vào hiệu suất chung của cầu thủ là gì?

# Chuẩn bị thư viện

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Khảo sát dữ liệu

Nạp dữ liệu vào bộ nhớ

In [4]:
df_match = pd.read_csv('./data/Match.csv')
df_player_attribute = pd.read_csv('./data/Player_Attributes.csv')
df_team_attribute = pd.read_csv('./data/Team_Attributes.csv')
df_team = pd.read_csv('./data/Team.csv')
df_player = pd.read_csv('./data/Player.csv')
df_country = pd.read_csv('./data/Country.csv')
df_league = pd.read_csv('./data/League.csv')

## Tập dữ liệu trận đấu

Lấy mẫu tập dữ liệu trận đấu

In [5]:
df_match.head() 

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


Các cột trong bảng trận đấu

In [6]:
df_match.columns 

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

Số lượng dòng và cột

In [7]:
df_match.shape

(25979, 115)

Thông tin data frame

In [8]:
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB
